<h1><center>Ray Tracing Project Pt. 2</center></h1>

Some basic imports:

In [ ]:
from tqdm.notebook import tqdm
import os
import time
import numpy as np
import matplotlib.pyplot as plt

The following function generates the world of choice given a key value. 
Worlds are organized in order of complexity.
Objects and textures are referenced here without explanation, check the .py files to see how each must be called if you are interested in generating your own object.

In [ ]:
#Import files from src folder
from pdf_src.objects import *
from src.textures import *
from src.bvh import *
from pdf_src.materials import *
def world_generator(key,speedup=True):
    """
    world_generator generates a 'world' for the program to render.

    :param key: This is an integer that acts as the key to a specific world
    :optional param speedup: This is a boolean value that determines whether 
                             the boundary box method should be used to speedup 
                             the search.
    :return: This function returns either a bvh node or a hittable_list, both of
             of which act as something that the ray_color function can send rays
             to. It also returns the length of the world, which is only used when
             plotting performance to track how the bvh speedup performs as a 
             function of the total number of objects in the world.
    """ 
    world = hittable_list()

    if key == 11:
        t0 = 0
        tf = 1
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        aluminum = metal(vec3(0.8,0.8,0.8),0.0)

        world.add_object(yz_rect(0,555,0,555,555,green)) #green
        world.add_object(yz_rect(0,555,0,555,0,red)) #red
        
        world.add_object(flip_face(xz_rect(213,343,227,332,554,light)))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        #world.add_object(xz_rect(213, 343, 227, 332, 554, light));
        box1 = box(vec3(0,0,0),vec3(165,330,165),aluminum)
        box1 = rotate_y(box1,25)
        box1 = translate(box1,vec3(265,0,295))
 
        world.add_object(box1)
        box2 = box(vec3(0,0,0),vec3(165,165,165),white)
        box2 = rotate_y(box2,-20)
        box2 = translate(box2,vec3(130,0,65)) 
        world.add_object(box2)
        

        
        
    elif key == 13:
        t0 = 0
        tf = 1
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        
        world.add_object(yz_rect(0,555,0,555,555,green))
        world.add_object(yz_rect(0,555,0,555,0,red))
        world.add_object(flip_face(xz_rect(213,343,227,332,554,light)))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        
        
        aluminum = metal(vec3(0.8,0.8,0.8),0.0)
        box1 = box(vec3(0,0,0),vec3(165,330,165),aluminum)
        box1 = rotate_y(box1,25)
        box1 = translate(box1,vec3(265,0,295))
        world.add_object(box1)
        
        glass = dielectric(1.5)
        world.add_object(sphere(vec3(190,90,190),90,glass))
        world.add_object(sphere(vec3(300,20,100),20,aluminum))

    elif key == 14:
        t0 = 0
        tf = 1
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        
        world.add_object(yz_rect(0,555,0,555,555,green))
        world.add_object(yz_rect(0,555,0,555,0,red))
        world.add_object(xz_rect(213,343,227,332,554,light))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        
        
        aluminum = metal(vec3(0.8,0.8,0.8),0.0)
        box1 = box(vec3(0,0,0),vec3(165,330,165),aluminum)
        box1 = rotate_y(box1,25)
        box1 = translate(box1,vec3(265,0,295))
        world.add_object(box1)
        
        # glass = dielectric(1.5)
        # world.add_object(sphere(vec3(190,90,190),90,glass))
        # world.add_object(sphere(vec3(320,40,100),40,aluminum))
        
    else:
        raise ValueError("Key must be 11 or 13")
        
    if speedup:
        bvh = bvh_node(world.data,0,len(world.data),t0,tf)
        final = bvh     
    else:
        final = world
    return final


In [ ]:
from src.camera import *
from src.ray import *
from pdf_src.image_writing import *
def main(key,aspectRatio,image_width,samps_per_pix,depth,speedup=True,write=True):
    """
    main: This is the driver function that actually performs the render

    :param key: This is an integer that acts as the key to a specific world.
    :param aspectRatio: The desired aspect ratio of the image
    :param image_width: The desired width of image to test on.
    :param samps_per_pix: This is the number of rays to send to each pixel
    :param depth: This is how far to follow each ray before we assume it disappears
    
    :optional: speedup: This defines whether the boundary box method is used at 
                        construction. It is a boolean value.
    :optional: write: This defines whether or not you want to write the image to
                      the image director. It is a boolean value. The only time
                      you'd really want this false is if you're measuring performance
                      and don't care about image output.
                        
    :return: The function returns the length of the world, mostly so the 
             measure_performance function can put that information on the plot.
    """   
    #Set up mage
    #default parameters, can change depending on key
    aspect_ratio = aspectRatio
    width = image_width #1200 #for full size image
    height = int(width/aspect_ratio)
    samples_per_pixel = samps_per_pix #increasing this will increase runtime
    max_depth = depth #how many times we'll follow a single ray before it returns no light
    
    
    
   
    
    #Generate world
    world = world_generator(key,speedup)
    
    
    
    
    #Initialize Camera
    
    #keys are matched with ray_tracing.ipynb for clarity of comparison
    if key == 11:
        background = vec3(0,0,0)
        lookfrom = vec3(278,278,-800)
        lookat = vec3(278,278,0)
        vup = vec3(0,1,0)
        vfov = 40
        dist_to_focus = 10
        aperture = 0

    elif key == 13:
        background = vec3(0,0,0)
        lookfrom = vec3(278,278,-800)
        lookat = vec3(278,278,0)
        vup = vec3(0,1,0)
        vfov = 40
        dist_to_focus = 10
        aperture = 0

    #set up materials to sample
    if key == 11:
        lights = hittable_list()
        lights.add_object(xz_rect(213,343,227,332,554,material()))
        
    elif key == 13:
        lights = hittable_list()
        lights.add_object(xz_rect(213,343,227,332,554,material()))
        lights.add_object(sphere(vec3(190,90,190),90,material()))
        
        
    cam = camera(lookfrom,lookat,vup,vfov,aspect_ratio,aperture,dist_to_focus,0,1)
    
    
    
    #Render the image
    lines = []
    for j in tqdm(range(height,0,-1)):
        for i in range(width):
            pixel_color = vec3(0.,0.,0.)
            for s in range(samples_per_pixel):
                u = (i + random.random())/(width-1)
                v = (j+random.random())/(height-1)
                r = cam.get_ray(u,v)
                pixel_color += ray_color(r,background,world,lights,max_depth)
            line = write_color(pixel_color,samples_per_pixel)
            lines.append(line)
    if write:
        image = open(f"./renders/pdf_world_{key}.ppm", "w")
        header = "".join(["P3\n",str(width),' ',str(height),"\n255\n"])
        image.write(header)
        image.writelines(lines)
        image.close()

        #Note: image magick is a required to convert the image to a jpeg
        #installation is straightforward with macports or homebrew (brew install imagemagick)
        #if you do not have image magick, the image will output as a PPM file
        #set convert to 0 if you are okay with that and don't want to install image magick
        convert = 1
        if convert == 1:
            to_jpg = f'convert ./renders/pdf_world_{key}.ppm ./renders/pdf_world_{key}.jpg'
            os.system(to_jpg)
            os.remove(f'./renders/pdf_world_{key}.ppm')
    

Suggested Aspect Ratio is 1:1. Only 2 keys are allowed here: 11 or 13

In [ ]:
width = 400
aspectRatio = 1
key = 4
depth = 50
speedup=False
samples_per_pix=200
write = True
main(key,aspectRatio,width,samples_per_pix,depth,speedup,write)



In [ ]:
width = 400
aspectRatio = 1
key = 13
depth = 50
speedup=False
samples_per_pix=200
write = True
main(key,aspectRatio,width,samples_per_pix,depth,speedup,write)



Now to see the time performance of the bounding box method, check performance.ipynb